In [1]:
import numpy as np
from glob import glob
from tqdm import tqdm
import h5py
import nrrd
import os
import pandas as pd
from dataset_split import remove_files
import SimpleITK as sitk

output_size =[128, 128, 64]

def resample_image3D(
    image3D,
    newspacing=[0.3,0.3,3],
    newsize=None,
    method='Linear',):
    """做插值"""
    resample = sitk.ResampleImageFilter()
    if method is 'Linear':
        resample.SetInterpolator(sitk.sitkLinear)
    elif method is 'Nearest':
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    resample.SetOutputDirection(image3D.GetDirection())
    resample.SetOutputOrigin(image3D.GetOrigin())
    resample.SetOutputSpacing(newspacing)

    if not newsize:
        newsize = np.round(np.array(image3D.GetSize())*np.abs(image3D.GetSpacing())/np.array(newspacing)).astype('int').tolist()

    resample.SetSize(newsize)
    # resample.SetDefaultPixelValue(0)

    newimage = resample.Execute(image3D)
    return newimage

# 数组替换元素
def array_replace(array,olds,news):
    #olds:list of old value
    #news:list of new value
    olds = np.array(olds)
    news = np.array(news)
    offset = olds.max()*10
    tmps = olds+offset
    for old,tmp in zip(olds,tmps):
        array[array==old] = tmp
    for tmp,new in zip(tmps,news):
        array[array==tmp] = new
    return array

def covert_h5(glob_str, old_replaced, new_replaced):
    """
    备注：不要骨头，骨头合并到背景类别中
    """
    listt = glob(glob_str)
    error_samples = []
    stats = pd.DataFrame(columns=['sample_name',
                                  'mean_whole', 
                                  'mean_bg', 
                                  'mean_dura', 
                                  'mean_SC', 
                                  'std_whole',
                                  'std_bg',
                                  'std_dura',
                                  'std_SC',
                                  'old_space0','old_space1','old_space2',
                                  'new_space0','new_space1','new_space2',
                                 ])
    for item in tqdm(listt):
        sample_name = item.split('/')[-2]
        print(sample_name,':')#win系统改为'\\'

        image = sitk.ReadImage(item)
        label = sitk.ReadImage(item.replace(old_replaced, 'Segmentation-label.nrrd'))
        seg = sitk.ReadImage(item.replace(old_replaced, 'Segmentation.seg.nrrd'))
        
        oldspacing = np.abs(image.GetSpacing())
        newspacing = [0.3, 0.3, 3.0]
        offset = [int(k) for k in seg.GetMetaData('Segmentation_ReferenceImageExtentOffset').split()]
        offset = np.round(np.array(offset)*np.array(oldspacing)/np.array(newspacing)).astype(int).tolist()
        label_name = [
            seg.GetMetaData('Segment0_Name'),
            seg.GetMetaData('Segment1_Name'),
            seg.GetMetaData('Segment2_Name') 
            ]#人工标注的类别顺序
        
        # resample
        image = resample_image3D(image,newspacing,method='Linear')
        label = resample_image3D(label,newspacing,method='Nearest')##,newsize=image.GetSize()
        seg = resample_image3D(seg,newspacing)
#         sizes = seg.GetSize()#resample后seg的size
        image = sitk.GetArrayFromImage(image).transpose((2,1,0))#tanspose之后才能与sizes匹配
        label = np.round( sitk.GetArrayFromImage(label) ).transpose((2,1,0)).astype(int)#tanspose之后才能与sizes匹配
#         seg = sitk.GetArrayFromImage(seg).transpose((3,2,1,0))#tanspose之后才能与sizes匹配
        
        
        # 灰度标准化
        image = (image - np.mean(image)) / np.std(image)
        image = image.astype(np.float32)        
        
#         # offset
#         image = image[offset[0]:offset[0]+sizes[0],
#                       offset[1]:offset[1]+sizes[1],
#                       offset[2]:offset[2]+sizes[2]]
#         label = label[offset[0]:offset[0]+sizes[0],
#                       offset[1]:offset[1]+sizes[1],
#                       offset[2]:offset[2]+sizes[2]].astype(np.uint8)
        
        # 错误病例：标记的尺寸和image尺寸不同，缺少其中一个类别或者多个类别的标记
        if not np.unique(label).tolist()==[0,1,2,3]:
            error_samples.append(sample_name)
            print("error sample(no dura/SC):",sample_name)
            continue
        
        #if not np.abs(np.array(image.shape)-np.array(seg.shape[1::])).sum()<=1:
        if not image.shape == label.shape:
            error_samples.append(sample_name)
            print("error sample(mismatch shape of image and label):",sample_name)
            continue
            
        
        ## 调整类别顺序，注意：seg是onehot编码
        target_name = ['dura','bone','SC']#目标类别顺序
        idx = [label_name.index(name) for name in target_name]
        idx = np.array(idx)+1
        idx = [0]+idx.tolist()
        label = array_replace(label,olds=[0,1,2,3],news=idx)
        
        # 合并骨头到背景中
        label[label==2] = 0
        label[label==3] = 2
        
        print("uncut image.shape:",image.shape,"uncut label.shape:",label.shape)
        assert np.unique(label).tolist()==[0,1,2], print('pixel classes are not [0,1,2]') 
        
        # cut
        tempL = np.nonzero(label)
        minx, maxx = np.min(tempL[0]), np.max(tempL[0])
        miny, maxy = np.min(tempL[1]), np.max(tempL[1])
        minz, maxz = np.min(tempL[2]), np.max(tempL[2])
        w, h, d = label.shape
        px = max(output_size[0] - (maxx - minx), 0) // 2
        py = max(output_size[1] - (maxy - miny), 0) // 2
        minx = max(minx - np.random.randint(10, 20) - px, 0)
        maxx = min(maxx + np.random.randint(10, 20) + px, w)
        miny = max(miny - np.random.randint(10, 20) - py, 0)
        maxy = min(maxy + np.random.randint(10, 20) + py, h)
        
        image = image[minx:maxx, miny:maxy]
        label = label[minx:maxx, miny:maxy]
        print("cut image.shape:",image.shape, "cut label.shape:",label.shape)
        
        f = h5py.File(item.replace(old_replaced, new_replaced), 'w')
        f.create_dataset('image', data=image, compression="gzip")
        f.create_dataset('label', data=label, compression="gzip")
        f.close()
    print("total number of seg-samples:", len(listt))
    return error_samples

def covert_h5_unseg(glob_str, old_replaced, new_replaced):
    """
    备注：无标注数据的格式转换
    """
    listt = glob(glob_str)
    for item in tqdm(listt):
        sample_name = item.split('/')[-2]
        print(sample_name,':')#win系统改为'\\'
        image = sitk.ReadImage(item)
        
        # resample
        newspacing = [0.3, 0.3, 3.0]
        image = resample_image3D(image,newspacing,method='Linear')
        image = sitk.GetArrayFromImage(image).transpose((2,1,0))#tanspose之后才能与sizes匹配
        
        # 灰度标准化
        image = (image - np.mean(image)) / np.std(image)
        image = image.astype(np.float32)
        print("image shape:",image.shape)
        
        f = h5py.File(item.replace(old_replaced, new_replaced), 'w')
        f.create_dataset('image', data=image, compression="gzip")
        f.close()  
    print("total number of unseg-samples:", len(listt))

    
if __name__ == '__main__':
    # 有标签数据
    print('seg dataset:')
    ## 先删除旧文件
    dataset_dir = '../../data/CTM_dataset/Segmented'
    re = os.path.join(dataset_dir,'*/mri_norm2.h5')
    remove_files(re=re)
    ## 再生成新文件
    glob_str = '../../data/CTM_dataset/Segmented/*/CTM.nrrd'
    error_samples=covert_h5(glob_str,'CTM.nrrd','mri_norm2.h5')
    
    # 无标签数据
    print('unseg dataset:')
    ## 先删除旧文件
    dataset_dir = '../../data/CTM_dataset/unSegmented'
    re = os.path.join(dataset_dir,'*/mri_norm2.h5')
    remove_files(re=re)
    ## 再生成新文件
    glob_str = '../../data/CTM_dataset/unSegmented/*/CTM.nrrd'
    covert_h5_unseg(glob_str,'CTM.nrrd','mri_norm2.h5')
    glob_str = '../../data/CTM_dataset/unSegmented/*/CT-vol.nrrd'
    covert_h5_unseg(glob_str,'CT-vol.nrrd','mri_norm2.h5')      


  0%|          | 0/181 [00:00<?, ?it/s]

seg dataset:
845929 :
image shape: (799, 799, 92)
uncut image.shape: (799, 799, 92)
uncut label.shape: (799, 799, 92)
cut image.shape: (159, 185, 92)
cut label.shape: (159, 185, 92)


  1%|          | 1/181 [00:10<31:17, 10.43s/it]

1559027 :
image shape: (492, 492, 73)
uncut image.shape: (492, 492, 73)
uncut label.shape: (492, 492, 73)
cut image.shape: (157, 182, 73)
cut label.shape: (157, 182, 73)


  1%|          | 2/181 [00:13<24:32,  8.23s/it]

...

total number of unseg-samples: 212
1279071 :
image shape: (478, 478, 72)


 50%|█████     | 1/2 [00:01<00:01,  1.59s/it]

1755610 :
image shape: (671, 671, 85)


100%|██████████| 2/2 [00:05<00:00,  2.73s/it]

total number of unseg-samples: 2


# 数据集划分&生成*.list文件

In [2]:
from dataset_split import dataset_split, make_dataset_list
# 有标签数据
save_dir = '../../data/CTM_dataset'
dataset_dir = '../../data/CTM_dataset/Segmented'
list_train,list_test = dataset_split(path=dataset_dir,save_dir=save_dir)
# 无标签数据
dataset_dir = '../../data/CTM_dataset/unSegmented'
make_dataset_list(path=dataset_dir,save_dir=save_dir)

In [3]:
error_samples

['1709234', '1409022-no SC', 'B1709234', 'B1334915-need revision', 'B1409022']

In [4]:
list_train

['B1272415',
 '1756747',
 '1685560',
 '1227957',
 '1433061',
 'B840383',
 'B1328036',
 'B1182723',
 '945286',
 '1261721',
 'B976555',
 'B1424004',
 '1775169',
 '1433611',
 '1247642',
 '1198882',
 '772230',
 '1460036-thoracic',
 'b1700637-neck',
 '1650523',
 'b1729235-noexcel',
 '928647',
 'b824280-dingzi',
 'B805736',
 '976555',
 'B1756747',
 'B820056',
 '1713865',
 '602788',
 '634961',
 'B1425167',
 '970380',
 'b1460036-neck',
 '1187634',
 '1204183-dingzi',
 'b930124',
 'B1713865',
 '797018',
 '845929',
 'B1536553',
 'B843043',
 'b1433611',
 'B1448326',
 '824280-dingzi',
 'b936932-dingzi',
 'B1302098',
 'b1071084',
 'B1694915',
 'B1559027',
 '1546477',
 'B1362738',
 '1700637',
 'b1460036-thoracic',
 'B1352900',
 'B1227957',
 'B1027338',
 'B617529',
 '643486',
 '759663',
 'B669349',
 'B685245',
 'B1632110',
 'B945286-dingzi',
 'B1532418',
 'B634961',
 '1362738',
 'B652658',
 'B1049094',
 '1302098',
 'B1433061',
 '1424004',
 'B1658899',
 'B820069',
 '987535',
 '1333768',
 'B868763',
 'B